# Model Optimization

In [1]:
### PACKAGES IMPORTS ###
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tcn import TCN
import optuna
from optuna import Trial
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

### INCLUDE GRAPHS IN NOTEBOOK AUTOMATICALLY ###
%matplotlib inline 

## Import Data

In [2]:
#load csv file
raw_data = pd.read_csv('C:/Users/nghj7/Desktop/Flight data/filtered_data.csv');
#create dataframe
df = pd.DataFrame(raw_data);
print('Shape of dataframe: ', df.shape)
df

Shape of dataframe:  (6832, 22)


,time,flight,drone_type,v_N,v_E,v_D,a_N,a_E,a_D,wind_N,...,battery_power,alt,long,lat,phi,theta,psi,angular_x,angular_y,angular_z
0,0.000,1,matrice_100,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,...,0.000000,269.332402,-79.782396,40.458047,0.000560,0.009941,-0.516789,-0.047280,0.080640,0.390483
1,0.810,1,matrice_100,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,...,0.000000,269.335937,-79.782396,40.458047,0.000561,0.009950,-0.516782,-0.232598,0.058941,-0.145898
2,1.500,1,matrice_100,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,...,0.000000,269.332274,-79.782397,40.458047,0.000571,0.009934,-0.516757,0.030549,0.091090,-0.358082
3,2.300,1,matrice_100,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,...,0.000000,269.331324,-79.782397,40.458047,0.000576,0.009938,-0.516745,-0.038736,0.151272,0.349120
4,3.000,1,matrice_100,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,...,0.000000,269.330575,-79.782397,40.458048,0.000587,0.009939,-0.516732,0.081788,0.049108,0.186339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6827,445.858,501,inspire,-0.016295,-0.047314,0.923826,0.005702,-0.013041,-1.065106,0.0,...,342.151992,92.801610,103.683285,1.344939,1.523057,-0.082441,-151.844488,8.974604,-5.662300,-3.522902
6828,446.858,501,inspire,0.025509,-0.027040,0.356245,-0.002888,-0.006615,-1.101281,0.0,...,340.677040,92.021736,103.683285,1.344939,0.956871,-0.789295,-155.280442,-1.846924,0.216854,-3.134087
6829,447.858,501,inspire,0.039037,-0.101185,0.012075,0.004992,-0.002673,-1.033789,0.0,...,340.677040,91.849470,103.683284,1.344938,0.323613,1.214056,-151.855352,-0.667574,-1.356790,10.510001
6830,448.858,501,inspire,0.003986,-0.048694,0.073468,0.005926,-0.002943,-1.054360,0.0,...,387.050224,91.655090,103.683283,1.344938,-0.770386,0.245185,-149.771986,20.306923,4.552239,-0.612599


## Model Variables

In [3]:
features =  ['v_N','v_E','v_D',
             'a_N','a_E','a_D',
             'wind_N', 'wind_E',
             'total_mass']
labels = ['battery_power']
test_seed = 1111
valid_seed = 2222

## Recurrent Neural Network Preprocessing

In [4]:
def custom_data_split(dataframe, time_step):
    """
    This function takes the filtered dataframe, time step n as argument.
    Returns the train, test, validation and drone list. 
    
    Output dimension will be (BATCH, TIMESTEP, FEATURES)
    Pre-padding for all earlier time steps: [-1e9, -1e9, 0, 1, 2, 3..]
    Post-padding to make sure BATCH is divisible by batch size
    """    
    flight_list = np.unique(dataframe['flight'])
    x_data = []
    for flno in flight_list:
        #filter flight no
        temp = dataframe[dataframe['flight'] == flno].copy()
        x_temp = []
        #apply pre-padding to TIME dimension
        for i in range(1,time_step+1):
            sliced = temp.iloc[0:i].copy()
            x_slice = sliced[features].values.tolist()
            x_padding = [-1e9] * len(features)
            x_slice = [x_padding]*(time_step-i) + x_slice
            x_temp.append(x_slice)
        #remaining time step
        remaining = [temp[i-time_step:i] for i in range(time_step, temp.shape[0])]
        x_temp += [i[features].values.tolist() for i in remaining if i.shape[0] == time_step]
        x_data.append(x_temp)
    #reshape
    x_data, y_data, all_drones = np.vstack(x_data), dataframe[labels], dataframe['drone_type']
    #train-test split
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, stratify = all_drones, random_state = test_seed)
    remaining_drones = all_drones.iloc[y_train.index]
    #train-validation split
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, stratify = remaining_drones, random_state = valid_seed) 
    #reshape
    y_train, y_valid, y_test = y_train.to_numpy(), y_valid.to_numpy(), y_test.to_numpy()
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test

#test function
x_train, x_valid, x_test, y_train, y_valid, y_test = custom_data_split(df, 60)
print('X_train shape:', x_train.shape)
print('Y_train shape:', y_train.shape)
print('X_valid shape:', x_valid.shape)
print('Y_valid shape:', y_valid.shape)
print('X_test shape:', x_test.shape)
print('Y_test shape:', y_test.shape)

X_train shape: (4372, 60, 9)
Y_train shape: (4372, 1)
X_valid shape: (1093, 60, 9)
Y_valid shape: (1093, 1)
X_test shape: (1367, 60, 9)
Y_test shape: (1367, 1)


## Functions to Create and Train Model

In [5]:
def create_model(model_name, params):
    """
    Create the model based on params as a dictionary.
    Returns the instantiated model
    """
    #MULTILAYER PERCEPTRON
    if model_name == 'mlp':
        #create input layer
        input_shape = (len(features),)
        model =  Sequential([tf.keras.Input(shape = input_shape), 
                             BatchNormalization()])
        #define kernel regularizer      
        reg = tf.keras.regularizers.L1L2(l1 = params['reg_alpha'], l2 = params['reg_lambda'])
        #add hidden layers
        dropped = False
        for i in range(params['n_layers']):
            model.add(Dense(units = params['n_units_' + str(i)], activation = 'relu', kernel_regularizer = reg))
            #add dropout layer
            if not dropped and params['n_layers'] != 1: 
                model.add(Dropout(params['dropout_rate']))
                dropped = True        
        #output layer  
        model.add(Dense(units=1, activation = 'linear'))
        #compile model
        opt = tf.keras.optimizers.Adam(learning_rate = params['learning_rate'])
        loss = params['loss']
        metric = params['metric']
        model.compile(optimizer = opt, loss = loss, metrics = [metric])
        
    #RECURRENT NETWORK
    elif model_name in ['lstm', 'tcn']:
        #define batch size, time step and num layers
        batch_size = 128
        time_step = params['time_step']
        n_layers = params['n_layers']
        #create input layer
        batch_input_shape = (batch_size, time_step, len(features)-1)
        time_input = tf.keras.Input(batch_shape = batch_input_shape, name ='time_variant')
        fixed_input = tf.keras.Input(batch_shape = (batch_size,1), name = 'time_invariant')
    
        #masking layer
        x = tf.keras.layers.Masking(mask_value=-1e9)(time_input)
        x1 = tf.keras.layers.Masking(mask_value=-1e9)(fixed_input)
        #create TCN
        if model_name == 'tcn': 
            #define exponential dilation, kernel and filter size
            dilations = [2**i for i in range(n_layers)]
            kernel_size = params['kernel_size']
            filter_size =  params['filter_size']
            for i in range(n_layers):
                if i != n_layers-1:
                    #hidden layers
                    x = TCN(nb_filters = filter_size, 
                            dilations = dilations, 
                            use_layer_norm = True, 
                            kernel_size = kernel_size,
                            return_sequences = True)(x)
                else:
                    #last layer
                    x = TCN(nb_filters = filter_size,
                            dilations = dilations,
                            use_layer_norm = True,
                            kernel_size = kernel_size, 
                            return_sequences = False)(x)
        #create LSTM
        elif model_name == 'lstm':
            #define kernel regularizer
            reg = tf.keras.regularizers.L1L2(l1 = params['reg_alpha'], l2 = params['reg_lambda'])
            for i in range(n_layers):
                if i != n_layers-1:
                    #hidden layers
                    x = tf.keras.layers.LSTM(units = params['n_units_'+str(i)], return_sequences = True, kernel_regularizer= reg)(x)
                else:
                    #last layer
                    x = tf.keras.layers.LSTM(units = params['n_units_'+str(i)], kernel_regularizer = reg)(x)
        #concatenate time and non-time input
        x_concat = tf.keras.layers.concatenate([x, x1])
        #final dense layers
        y = tf.keras.layers.Dense(10, activation='relu')(x_concat)
        y = tf.keras.layers.Dense(1, activation= None)(y)
        #create model
        model = Model(inputs = [time_input, fixed_input], outputs = [y])
        #define optimizer, loss function and metric
        opt = tf.keras.optimizers.Adam(learning_rate = params['learning_rate'])
        loss = params['loss']
        metric = params['metric']
        #compile model
        model.compile(optimizer = opt, loss = loss, metrics = [metric])
        
    return model 

In [6]:
def train_model(model_name, model, params, x_train, x_valid, y_train, y_valid, trial):
    """
    Trains the model using train and validation set.
    Returns the trained model and loss history    
    """
    #MULTILAYER PERCEPTRON
    if model_name == 'mlp':
        #batch calculation
        batch_size = 128
        #training calclation
        n_train_examples = x_train.shape[0]
        epoch = 800
        steps_per_epoch = int(n_train_examples / batch_size) 
        #create train and validation set
        dtrain = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size, drop_remainder = True)
        dvalid = tf.data.Dataset.from_tensors((x_valid, y_valid))
        #create callbacks to fit method
        monitor = params['monitor']
        early_stopping = tf.keras.callbacks.EarlyStopping(mode='auto', patience = 20, monitor = monitor)
        learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor = monitor, factor=params['eta_decay'], 
                                                                       patience = 5, min_lr = 0.1)
        #######
        pruner =  optuna.integration.TFKerasPruningCallback(trial, monitor)
        #######
        callbacks = [learning_rate_reduction, early_stopping, pruner]
        #fit model
        history = model.fit(dtrain, epochs = epoch, steps_per_epoch = steps_per_epoch, 
                            callbacks = callbacks, validation_data = (x_valid, y_valid), verbose = 0)
    
    #RECURRENT NETWORKS
    elif model_name in ['lstm', 'tcn']:
        #batch calculation
        batch_size = 128 
        #training calculation
        n_train_examples = x_train.shape[0]
        epoch = 150
        steps_per_epoch = int(n_train_examples / batch_size)
        #create train set
        train_time_input = x_train[:, :, :-1]
        train_fixed_input = x_train[:, -1, -1].reshape(-1, 1)
        x_train = {'time_variant':train_time_input, 'time_invariant':train_fixed_input}
        dtrain = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size, drop_remainder = True)
        #create validation set
        valid_time_input = x_valid[:, :, :-1]
        valid_fixed_input = x_valid[:, -1, -1].reshape(-1, 1)
        x_valid = {'time_variant':valid_time_input, 'time_invariant':valid_fixed_input}
        dvalid = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size, drop_remainder = True)   
        #create callbacks to fit method
        monitor = params['monitor']
        early_stopping = tf.keras.callbacks.EarlyStopping(mode='auto', patience = 20, monitor = monitor)
        learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor = monitor, factor=params['eta_decay'], 
                                                                       patience =5, min_lr = 0.1)
        #######
        pruner =  optuna.integration.TFKerasPruningCallback(trial, monitor)
        #######
        callbacks = [learning_rate_reduction, early_stopping, pruner]    
        #fit model
        history = model.fit(dtrain, epochs = epoch, steps_per_epoch = steps_per_epoch, 
                            callbacks = callbacks, validation_data = dvalid, verbose = 0)
        
    return history

## Model Optimization 

In [7]:
def optuna_objective(trial: Trial):
    """
    This function takes the trial object from Optuna and uses the TPE algorithm
    to sample the hyperparameters. Returns the history of validation score to 
    Optuna for pruning.
    """
    params = {}
    #PARAMS
    if current_model == 'mlp':
        params['n_layers'] = trial.suggest_int('n_layers', 1, 3)
        params['reg_alpha'] = trial.suggest_float('reg_alpha', 1e-6, 1e-2, log=True)
        params['reg_lambda'] = trial.suggest_float('reg_lambda', 1e-6, 1e-2, log=True)
        params['dropout_rate'] = trial.suggest_float('dropout_rate', 0.1, 0.4, log=True)
        for i in range(params['n_layers']):
            params['n_units_'+str(i)] = trial.suggest_categorical("n_units_{}".format(i), [32, 64, 128, 256])
        params['learning_rate'] = trial.suggest_float('learning_rate', 1e-6, 1e-1, log = True)
        params['eta_decay'] = trial.suggest_float('eta_decay', 0.5, 0.9)
        #split train-test set
        all_drones = df['drone_type']
        x_train, x_test, y_train, y_test = train_test_split(df[features], df[labels], test_size = 0.2, stratify = all_drones, shuffle = True, random_state = test_seed)
        remaining_drones = all_drones.loc[x_train.index]
        #split validation set
        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, stratify = remaining_drones, shuffle = True, random_state = valid_seed)
        
    elif current_model == 'tcn':
        params['time_step'] = trial.suggest_int('time_step', 1, 40)
        params['n_layers'] = trial.suggest_int('n_layers', 2, 5)
        params['kernel_size'] = trial.suggest_int('kernel_size', 2, 4)
        params['filter_size'] = trial.suggest_categorical('filter_size', [16,32,64,128])
        params['learning_rate'] = trial.suggest_float('learning_rate', 1e-6, 1e-1, log = True)
        params['eta_decay'] = trial.suggest_float('eta_decay', 0.5, 0.9)
        #split train-test set
        x_train, x_valid, x_test, y_train, y_valid, y_test = custom_data_split(df, params['time_step'])
    elif current_model == 'lstm':
        params['time_step'] = trial.suggest_int('time_step', 1, 40)
        params['n_layers'] = trial.suggest_int('n_layers', 1, 3)
        params['reg_alpha'] = trial.suggest_float('reg_alpha', 1e-6, 1e-2, log=True)
        params['reg_lambda'] = trial.suggest_float('reg_lambda', 1e-6, 1e-2, log=True)
        for i in range(params['n_layers']):
            params['n_units_' + str(i)] = trial.suggest_categorical("n_units_{}".format(i), [16, 32, 64, 128])
        params['learning_rate'] = trial.suggest_float('learning_rate', 1e-6, 1e-1, log = True)
        params['eta_decay'] = trial.suggest_float('eta_decay', 0.5, 0.9)
        #split train-test set
        x_train, x_valid, x_test, y_train, y_valid, y_test = custom_data_split(df, params['time_step'])
        
    #OBJECTIVE
    if current_objective == 'mse':
        params['loss'] = tf.keras.losses.MeanSquaredError()
        params['metric'] = tf.keras.metrics.MeanSquaredError()
        params['monitor'] = 'val_mean_squared_error'
    elif current_objective =='quantile':
        params['loss'] = tfa.losses.PinballLoss(tau = quantile, name = 'quantile')
        params['metric'] = tfa.losses.PinballLoss(tau = quantile, name = 'quantile', reduction = tf.keras.losses.Reduction.NONE)
        params['monitor'] = 'val_quantile'
        
    #create model
    model = create_model(current_model, params)
    
    #train model
    history = train_model(current_model, model, params, x_train, x_valid, y_train, y_valid, trial)
    
    #get validation score
    valid_score = history.history[params['monitor']][-1]
    
    #return score to optuna
    return valid_score

In [8]:
model_name = ['mlp', 'lstm', 'tcn']
objective = ['mse', 'quantile']
quantile_list = [0.1, 0.5, 0.9]
quantile_study = {}

for model in model_name:
    for i, var in enumerate(quantile_list):
        current_objective = 'quantile'
        current_model = model
        quantile = var
        #create study (default sampler = TPE, objective = quantile) 
        pruner = optuna.pruners.MedianPruner(n_startup_trials = 5, n_warmup_steps =3)
        study = optuna.create_study(pruner=pruner, direction ='minimize')
        study.optimize(optuna_objective, n_trials = 50);
        quantile_study[model + 'quantile' + str(var)] = study

[I 2022-03-20 00:17:43,867] A new study created in memory with name: no-name-ed2eca8c-15bf-4ac3-8e19-8bd5bf2b6155
[I 2022-03-20 00:18:18,871] Trial 0 finished with value: 16.907493591308594 and parameters: {'n_layers': 1, 'reg_alpha': 0.0008179169587463382, 'reg_lambda': 6.410826140753051e-05, 'dropout_rate': 0.13469147625550457, 'n_units_0': 64, 'learning_rate': 0.0024135044592857595, 'eta_decay': 0.5998877876548145}. Best is trial 0 with value: 16.907493591308594.
[I 2022-03-20 00:18:53,972] Trial 1 finished with value: 17.42340660095215 and parameters: {'n_layers': 1, 'reg_alpha': 0.00011986210109864836, 'reg_lambda': 1.709287548216866e-05, 'dropout_rate': 0.14862648014077226, 'n_units_0': 128, 'learning_rate': 0.0005927430339979375, 'eta_decay': 0.7501937543811001}. Best is trial 0 with value: 16.907493591308594.
[I 2022-03-20 00:19:29,794] Trial 2 finished with value: 17.31326675415039 and parameters: {'n_layers': 1, 'reg_alpha': 0.0007317399061705723, 'reg_lambda': 0.003206513705

[I 2022-03-20 00:21:45,363] Trial 2 finished with value: 21.53697395324707 and parameters: {'n_layers': 3, 'reg_alpha': 0.005272998712870087, 'reg_lambda': 0.0008631549891994171, 'dropout_rate': 0.1910360937987662, 'n_units_0': 64, 'n_units_1': 32, 'n_units_2': 64, 'learning_rate': 0.0004647816363223246, 'eta_decay': 0.7452358321916969}. Best is trial 1 with value: 16.519969940185547.
[I 2022-03-20 00:22:33,273] Trial 3 finished with value: 68.61414337158203 and parameters: {'n_layers': 3, 'reg_alpha': 0.0015981222076715345, 'reg_lambda': 1.811304997180709e-06, 'dropout_rate': 0.11479087073895697, 'n_units_0': 256, 'n_units_1': 64, 'n_units_2': 64, 'learning_rate': 5.186324821878977e-06, 'eta_decay': 0.8955121244094748}. Best is trial 1 with value: 16.519969940185547.
[I 2022-03-20 00:22:53,412] Trial 4 finished with value: 24.425203323364258 and parameters: {'n_layers': 3, 'reg_alpha': 5.123130260887064e-06, 'reg_lambda': 0.0007683498290857007, 'dropout_rate': 0.2125591066755165, 'n_u

[I 2022-03-20 00:24:33,179] A new study created in memory with name: no-name-aed5149d-df68-4345-9671-b8ed8ed79f35
[I 2022-03-20 00:24:35,741] Trial 0 finished with value: 17.507600784301758 and parameters: {'n_layers': 1, 'reg_alpha': 0.009540825875257007, 'reg_lambda': 0.0032506194452602325, 'dropout_rate': 0.1643982229064812, 'n_units_0': 64, 'learning_rate': 0.05805101377545513, 'eta_decay': 0.832114089325541}. Best is trial 0 with value: 17.507600784301758.
[I 2022-03-20 00:25:16,706] Trial 1 finished with value: 331.4285583496094 and parameters: {'n_layers': 2, 'reg_alpha': 0.00016881792596506272, 'reg_lambda': 2.529283985680589e-05, 'dropout_rate': 0.12336741858483537, 'n_units_0': 32, 'n_units_1': 256, 'learning_rate': 4.756401516780025e-06, 'eta_decay': 0.7211538210647754}. Best is trial 0 with value: 17.507600784301758.
[I 2022-03-20 00:25:25,745] Trial 2 finished with value: 18.305408477783203 and parameters: {'n_layers': 1, 'reg_alpha': 7.949716958869124e-05, 'reg_lambda': 1

[I 2022-03-20 00:28:33,842] Trial 39 pruned. Trial was pruned at epoch 9.
[I 2022-03-20 00:28:34,650] Trial 40 pruned. Trial was pruned at epoch 5.
[I 2022-03-20 00:28:35,188] Trial 41 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:35,728] Trial 42 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:36,264] Trial 43 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:36,800] Trial 44 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:37,436] Trial 45 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:37,971] Trial 46 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:38,511] Trial 47 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:39,274] Trial 48 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:39,893] Trial 49 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 00:28:39,893] A new study created in memory with name: no-name-8281e6cf-b8c2-4e19-9397-538b1203bac5
[I 2022-03-20 00:31:57,280] Trial 0 finished with value: 41.374885559082

[I 2022-03-20 01:07:28,836] Trial 35 pruned. Trial was pruned at epoch 32.
[I 2022-03-20 01:07:37,871] Trial 36 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:08:01,168] Trial 37 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:08:14,316] Trial 38 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:08:41,953] Trial 39 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:09:01,176] Trial 40 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:09:26,206] Trial 41 pruned. Trial was pruned at epoch 6.
[I 2022-03-20 01:09:47,714] Trial 42 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:10:07,053] Trial 43 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:10:23,767] Trial 44 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:11:33,282] Trial 45 pruned. Trial was pruned at epoch 5.
[I 2022-03-20 01:11:51,533] Trial 46 pruned. Trial was pruned at epoch 9.
[I 2022-03-20 01:12:05,695] Trial 47 pruned. Trial was pruned at epoch 6.
[I 2022-03-20 01:13:03,424] Trial 48 

[I 2022-03-20 01:51:04,030] Trial 33 finished with value: 27.834400177001953 and parameters: {'time_step': 6, 'n_layers': 3, 'reg_alpha': 5.560574120581462e-05, 'reg_lambda': 0.00021064345096735815, 'n_units_0': 64, 'n_units_1': 64, 'n_units_2': 128, 'learning_rate': 0.028727757128373166, 'eta_decay': 0.7037942141012505}. Best is trial 2 with value: 17.07394027709961.
[I 2022-03-20 01:51:45,831] Trial 34 pruned. Trial was pruned at epoch 5.
[I 2022-03-20 01:51:53,431] Trial 35 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:52:51,425] Trial 36 pruned. Trial was pruned at epoch 12.
[I 2022-03-20 01:53:08,462] Trial 37 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:53:27,020] Trial 38 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:53:52,537] Trial 39 pruned. Trial was pruned at epoch 12.
[I 2022-03-20 01:54:04,371] Trial 40 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 01:54:48,913] Trial 41 finished with value: 22.54808235168457 and parameters: {'time_step': 3, 

[I 2022-03-20 02:21:33,372] Trial 17 pruned. Trial was pruned at epoch 39.
[I 2022-03-20 02:21:55,664] Trial 18 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 02:22:05,586] Trial 19 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 02:24:49,552] Trial 20 finished with value: 19.065217971801758 and parameters: {'time_step': 36, 'n_layers': 2, 'reg_alpha': 0.00015786370688207442, 'reg_lambda': 0.0027303346345052193, 'n_units_0': 64, 'n_units_1': 16, 'learning_rate': 0.027161892442890866, 'eta_decay': 0.8421787186532839}. Best is trial 12 with value: 6.740644931793213.
[I 2022-03-20 02:25:00,496] Trial 21 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 02:25:12,831] Trial 22 pruned. Trial was pruned at epoch 6.
[I 2022-03-20 02:25:24,120] Trial 23 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 02:25:35,085] Trial 24 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 02:26:38,927] Trial 25 finished with value: 6.290726184844971 and parameters: {'time_step': 29, 'n_layers': 1, '

[I 2022-03-20 03:34:35,866] Trial 24 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:35:19,966] Trial 25 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:35:28,846] Trial 26 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:35:56,887] Trial 27 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:36:07,215] Trial 28 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:36:50,159] Trial 29 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:37:15,623] Trial 30 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 03:39:02,124] Trial 31 finished with value: 5.9786295890808105 and parameters: {'time_step': 28, 'n_layers': 2, 'kernel_size': 4, 'filter_size': 16, 'learning_rate': 0.0025172594278320163, 'eta_decay': 0.6277167572972111}. Best is trial 14 with value: 5.654911994934082.
[I 2022-03-20 03:41:25,300] Trial 32 finished with value: 5.384769439697266 and parameters: {'time_step': 28, 'n_layers': 2, 'kernel_size': 4, 'filter_size': 16, 'learning_rate': 0.0046941111613938

[I 2022-03-20 07:43:58,350] Trial 31 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:44:06,272] Trial 32 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:44:14,405] Trial 33 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:44:44,697] Trial 34 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:45:31,760] Trial 35 finished with value: 17.844440460205078 and parameters: {'time_step': 21, 'n_layers': 2, 'kernel_size': 4, 'filter_size': 16, 'learning_rate': 0.038560052099249925, 'eta_decay': 0.7617641455917583}. Best is trial 12 with value: 10.207880020141602.
[I 2022-03-20 07:47:12,761] Trial 36 pruned. Trial was pruned at epoch 9.
[I 2022-03-20 07:47:28,986] Trial 37 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:47:43,231] Trial 38 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:47:52,434] Trial 39 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 07:48:28,505] Trial 40 pruned. Trial was pruned at epoch 5.
[I 2022-03-20 07:50:14,201] Trial 41 finished wit

[I 2022-03-20 10:26:00,285] Trial 24 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 10:26:17,050] Trial 25 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 10:26:24,184] Trial 26 pruned. Trial was pruned at epoch 4.
[I 2022-03-20 10:27:14,123] Trial 27 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 10:30:02,888] Trial 28 finished with value: 11.640640258789062 and parameters: {'time_step': 12, 'n_layers': 2, 'kernel_size': 4, 'filter_size': 128, 'learning_rate': 0.07330266203555588, 'eta_decay': 0.665461759190956}. Best is trial 23 with value: 7.019050598144531.
[I 2022-03-20 10:30:25,706] Trial 29 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 10:30:55,104] Trial 30 pruned. Trial was pruned at epoch 3.
[I 2022-03-20 10:34:04,651] Trial 31 finished with value: 7.584063529968262 and parameters: {'time_step': 10, 'n_layers': 2, 'kernel_size': 4, 'filter_size': 64, 'learning_rate': 0.03785718723504038, 'eta_decay': 0.6209317629507044}. Best is trial 23 with value: 7.01905059

In [13]:
for name, key in quantile_study.items():
    print('Name: \n \n', name)
    print('\n Best Loss: \n \n', key.best_trial.value)
    print('\n Hyperparameters: \n \n', key.best_trial.params)
    print('\n-------------------')

Name: 
 
 mlpquantile0.1

 Best Loss: 
 
 8.096294403076172

 Hyperparameters: 
 
 {'n_layers': 3, 'reg_alpha': 6.212514791077957e-06, 'reg_lambda': 2.326652392916294e-06, 'dropout_rate': 0.10456554658254263, 'n_units_0': 256, 'n_units_1': 128, 'n_units_2': 32, 'learning_rate': 0.0219363399910911, 'eta_decay': 0.8507050517321648}

-------------------
Name: 
 
 mlpquantile0.5

 Best Loss: 
 
 16.015689849853516

 Hyperparameters: 
 
 {'n_layers': 2, 'reg_alpha': 0.0003292624191114745, 'reg_lambda': 2.710160495505777e-05, 'dropout_rate': 0.29060862233655127, 'n_units_0': 128, 'n_units_1': 64, 'learning_rate': 0.004381503682976994, 'eta_decay': 0.6545573090226162}

-------------------
Name: 
 
 mlpquantile0.9

 Best Loss: 
 
 9.194951057434082

 Hyperparameters: 
 
 {'n_layers': 2, 'reg_alpha': 8.111842828761397e-06, 'reg_lambda': 8.485504099805394e-06, 'dropout_rate': 0.2635141652611792, 'n_units_0': 256, 'n_units_1': 256, 'learning_rate': 0.0008705444763254024, 'eta_decay': 0.7124389516